In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.hypotheses import make_branches_multi, score_new_branches, make_hypotheses_from_trees, generate_hypothesis_matrix

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [69]:
detections = pd.DataFrame()
detections["id"] = [0, 1, 2, 3, 4]
detections["time"] = [0, 1, 1, 3, 3]
detections["object_location"] = [0, 1, 2, 3, 4]

faunastep = 0.4
bg_prob = 0.2

In [70]:
detections

,id,time,object_location
0,0,0,0
1,1,1,1
2,2,1,2
3,3,3,3
4,4,3,4


In [71]:
probz = generate_hypothesis_matrix(detections, faunastep)

/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:59: RuntimeWarning: divide by zero encountered in true_divide
  # score is the existing + background prob
/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:59: RuntimeWarning: invalid value encountered in true_divide
  # score is the existing + background prob


In [72]:
# make hypotheses and scores for first timestep
scores = []
hyps = []
for i, row in detections[detections["time"] == 0].iterrows():
    hyps.append([[row["id"]]])
    scores.append(bg_prob)

In [101]:
def update_hyps_scores_multi(det, hyps, scores, probz, bg_prob):
    new_hyps = []
    new_scores = []

    detection_ids = dets["id"].to_list()

    for hyp_i, hyp in enumerate(hyps):
        existing_score = scores[hyp_i]

        for h in hyp:
            branches = make_branches_multi(h, detection_ids)
            br_scores = score_new_branches(branches, bg_prob, probz)

            hyp_inds = make_hypotheses_from_trees(branches)

            for hind in hyp_inds:
                new_hyps.append([branches[i] for i in hind])

                new_scores.append(existing_score + np.sum([br_scores[i] for i in hind]))

    return new_hyps, new_scores
    

In [84]:
dets = detections[detections.time == 1]
detection_ids = dets["id"].to_list()

In [105]:
detection_ids

[1, 2]

In [85]:
hyps

[[[0]]]

In [106]:
hyps[0]

[[0]]

In [111]:
make_branches_multi(new_hyps[0], detection_ids)

[[0, nan, nan],
 [nan, 1, nan],
 [nan, 2, nan],
 [nan, nan, nan, 1],
 [nan, nan, nan, 2],
 [0, nan, 1],
 [0, nan, 2],
 [nan, 1, 1],
 [nan, 1, 2],
 [nan, 2, 1],
 [nan, 2, 2]]

In [103]:
branches = make_branches_multi(hyps[0], detection_ids)
br_scores = score_new_branches(branches, bg_prob, probz)

In [104]:
branches

[[0, nan], [nan, 1], [nan, 2], [0, 1], [0, 2]]

In [87]:
br_scores

[0, 0.2, 0.2, 0.012419330651552318, 5.733031438470704e-07]

In [88]:
hyp_inds = make_hypotheses_from_trees(branches)


In [89]:
new_hyps = []
new_scores = []

for hind in hyp_inds:
    new_hyps.append([branches[i] for i in hind])
    new_scores.append(scores[0] + np.sum([br_scores[i] for i in hind]))
    

In [90]:
new_hyps

[[[0, nan], [nan, 1], [nan, 2]], [[nan, 1], [0, 2]], [[nan, 2], [0, 1]]]

In [91]:
detection_ids

[1, 2]

In [92]:
for i, h in enumerate(new_hyps):
    print(h, new_scores[i])

[[0, nan], [nan, 1], [nan, 2]] 0.6000000000000001
[[nan, 1], [0, 2]] 0.40000057330314387
[[nan, 2], [0, 1]] 0.41241933065155234


In [110]:
new_hyps[0]

[[0, nan], [nan, 1], [nan, 2]]

In [59]:
from src.hypotheses import update_hyps_scores

In [97]:
hyps

[[[0]]]

In [98]:
dets

,id,time,object_location
1,1,1,1
2,2,1,2


In [102]:
update_hyps_scores_multi(dets,
                  hyps,
                  scores,
                  probz,
                  bg_prob)

/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  trees = np.array(trees)


([[array([nan]), [nan, 1], [nan, 2], array([1]), array([2])]], [1.0])